# 📋 Démonstration NER + RAG pour Fiches de Défauts

Ce notebook démontre l'utilisation du système de NER hybride et d'intégration RAG pour extraire et compléter automatiquement les fiches de défauts.

## 🔧 Configuration et imports

In [ ]:
import sys
sys.path.append('../src')

from ner_defaut_documents import (
    extract_entities_from_defaut_document,
    generate_rag_completion_prompt,
    display_entities
)
from rag_integration_ner import DefautDocumentRAG, process_document_with_rag_suggestions

import json
from pathlib import Path
from IPython.display import display, Markdown, HTML

## 📄 Étape 1 : Charger un document OCR

In [ ]:
# Chemin vers le document exemple
doc_path = Path("../data/ocr_results/2291 - GAEC DE VAULEON - DEFAUT_ocr.txt")

# Lire le contenu
with open(doc_path, 'r', encoding='utf-8') as f:
    ocr_text = f.read()

print("📄 Document chargé:")
print("=" * 80)
print(ocr_text)
print("=" * 80)

## 🤖 Étape 2 : Extraction des entités avec NER

In [ ]:
# Extraire les entités
print("🔍 Extraction en cours...\n")
entities = extract_entities_from_defaut_document(ocr_text)

# Afficher les résultats
display_entities(entities)

## 📊 Étape 3 : Visualiser les données extraites

In [ ]:
# Afficher le JSON structuré
print("📊 Données structurées (JSON):\n")
print(json.dumps(entities, indent=2, ensure_ascii=False))

## 📋 Étape 4 : Analyse des champs manquants

In [ ]:
champs_manquants = entities.get('champs_manquants', [])

if champs_manquants:
    display(Markdown(f"### ⚠️ {len(champs_manquants)} champ(s) manquant(s)"))
    
    for i, champ in enumerate(champs_manquants, 1):
        print(f"{i}. {champ}")
else:
    display(Markdown("### ✅ Tous les champs sont remplis !"))

## 💬 Étape 5 : Génération du prompt RAG

In [ ]:
# Générer le prompt de complétion
rag_prompt = generate_rag_completion_prompt(entities)

display(Markdown(rag_prompt))

## 🎯 Étape 6 : Initialiser le système RAG

In [ ]:
# Initialiser le RAG
rag = DefautDocumentRAG(ocr_text)

# Obtenir le prompt initial
initial_prompt = rag.get_initial_prompt()

display(Markdown("### 🤖 Prompt initial du RAG"))
display(Markdown(initial_prompt))

## 💭 Étape 7 : Simulation d'un dialogue RAG

In [ ]:
# Premier message utilisateur
user_message_1 = "Le numéro d'AO est AO-2022-0456"

display(Markdown(f"**👤 Utilisateur:** {user_message_1}"))

response_1 = rag.chat(user_message_1)

display(Markdown(f"**🤖 Assistant:** {response_1}"))

In [ ]:
# Deuxième message utilisateur
user_message_2 = "Oui, le document a été signé"

display(Markdown(f"**👤 Utilisateur:** {user_message_2}"))

response_2 = rag.chat(user_message_2)

display(Markdown(f"**🤖 Assistant:** {response_2}"))

## 📊 Étape 8 : Suggestions RAG avancées

In [ ]:
# Obtenir les suggestions RAG détaillées
result_with_suggestions = process_document_with_rag_suggestions(str(doc_path))

display(Markdown("### 🎯 Questions suggérées pour le RAG"))

for question in result_with_suggestions['rag_suggestions']['questions_a_poser']:
    display(Markdown(f"**Champ:** `{question['champ']}`"))
    display(Markdown(f"**Question:** {question['question']}"))
    display(Markdown(f"**Type:** {question['type']}"))
    
    if 'suggestions' in question:
        display(Markdown(f"**Suggestions:** {', '.join(question['suggestions'])}"))
    
    print("\n---\n")

## 📈 Étape 9 : Statistiques et qualité

In [ ]:
# Calculer les statistiques
mes = entities.get('mise_en_service', {})
champs_remplis_mes = sum(1 for v in mes.values() if v)
total_champs_mes = len(mes)

tableau = entities.get('tableau_defauts', [])
lignes_completes = sum(1 for item in tableau if item.get('anomalies') and item.get('temps_passe'))
total_lignes = len(tableau)

qualite_ocr = entities.get('qualite_ocr', 'inconnue')

# Afficher les statistiques
stats_html = f"""
<div style="background-color: #f0f0f0; padding: 20px; border-radius: 10px;">
    <h3>📊 Statistiques du document</h3>
    
    <div style="margin: 10px 0;">
        <strong>Mise en Service:</strong><br>
        Champs remplis: {champs_remplis_mes} / {total_champs_mes} 
        ({champs_remplis_mes/total_champs_mes*100:.1f}%)
    </div>
    
    <div style="margin: 10px 0;">
        <strong>Tableau des Défauts:</strong><br>
        Lignes complètes: {lignes_completes} / {total_lignes} 
        ({lignes_completes/total_lignes*100:.1f}%)
    </div>
    
    <div style="margin: 10px 0;">
        <strong>Qualité OCR:</strong> 
        <span style="color: {'green' if qualite_ocr == 'bonne' else 'orange' if qualite_ocr == 'moyenne' else 'red'};">
            {qualite_ocr.upper()}
        </span>
    </div>
    
    <div style="margin: 10px 0;">
        <strong>Champs manquants:</strong> {len(champs_manquants)}
    </div>
</div>
"""

display(HTML(stats_html))

## 💾 Étape 10 : Export des données

In [ ]:
# Sauvegarder les résultats
output_dir = Path("../data/ner_results")
output_dir.mkdir(exist_ok=True)

output_file = output_dir / "demo_extraction_result.json"

export_data = {
    "document_source": str(doc_path),
    "entites_extraites": entities,
    "prompt_rag": rag_prompt,
    "suggestions": result_with_suggestions['rag_suggestions']
}

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

display(Markdown(f"### ✅ Données exportées vers: `{output_file}`"))

## 🎓 Conclusion

Ce notebook a démontré:

1. ✅ **Extraction NER** - Utilisation d'un LLM pour extraire les entités structurées
2. ✅ **Identification des manques** - Détection automatique des champs incomplets
3. ✅ **Génération de prompts RAG** - Création de questions intelligentes
4. ✅ **Dialogue interactif** - Simulation d'une conversation pour compléter le document
5. ✅ **Statistiques et qualité** - Évaluation de la complétude et de la qualité OCR

### 🚀 Prochaines étapes

- Intégrer dans l'interface Streamlit
- Créer une API REST pour l'automatisation
- Ajouter une base de données pour stocker les documents complétés
- Implémenter un système de validation multi-niveaux